# Installs

In [ ]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 42.2 MB/s eta 0:00:00



This may take a while

In [ ]:
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

!pip install torchsummaryX==1.3.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 25.0 MB/s eta 0:00:00
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 8.41 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.c

In [ ]:
'''
If torchsummaryX doesn't work, please run this cell. Alternatively, please refer to Piazza post @209 for more assistance:
'''

# !pip install torchsummaryx====1.3.0

"\nIf torchsummaryX doesn't work, please run this cell. Alternatively, please refer to Piazza post @209 for more assistance:\n"

# Imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime
import glob
import pdb

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Handout Notes:

- This problem is hard because it is order-aligned but not time-synchronous
  - It is not immediately obvious where the sequence "Y\EH\S" would align to each cell of a recurrent network
- CTC is used for this reason. Instead of relying on the network to produce outputs at erratic times, you have the network produce outputs at every time and then perform a DP solution to search over the output space.
- Model Notes:
  - Generally:
    - Use GRU/LSTM
    - Model can be bidirectional since we have the whole input sequence at test time
    - Downsampling the input sequence could be advantageous and is realistic since an audio file will often repeat phenomes over multiple time steps
  - Specifically:
    - 1D CNN can extract latent features
    - try bi-LSTMs or pBiLSTMs

# TODOs

# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8 -q
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"alectrela","key":"42d0f6df23d52bb3b9e5a7b2cb5e1962"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!kaggle competitions download -c hw3p2asr-s24

100% 3.74G/3.74G [01:38<00:00, 36.4MB/s]
100% 3.74G/3.74G [01:38<00:00, 40.7MB/s]


In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-s24-hw3p2  ctcdecode  hw3p2asr-s24.zip  sample_data
'''
!unzip -q hw3p2asr-s24.zip
!ls

11-785-s24-hw3p2  ctcdecode  hw3p2asr-s24.zip  sample_data


# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [ ]:
root = '/content/11-785-s24-hw3p2/'

# Feel free to add more items here
config = {
    "model"      : "transforms-1",
    "beam_width" : 10,
    "lr"         : 1.5e-3,
    "epochs"     : 50,
    "batch_size" : 128,  # Increase if your device can handle it
    "milestones": [15, 20, 25, 30, 35, 40, 45],
    "gamma"      : 0.4,
    "freq_mask" : random.randint(35, 65),
    "time_mask" : random.randint(90, 150),
    "time_p": (random.random() * 0.35) + 0.25,
    "embed_size": 220
}

# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
transforms = [
    tat.FrequencyMasking(freq_mask_param=config["freq_mask"]),
    tat.TimeMasking(time_mask_param=config["time_mask"], p=config["time_p"])
 ] # set of tranformations

In [ ]:
# You might want to play around with the mapping as a sanity check here

### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root, phonemes=PHONEMES, partition="train-clean-100", transforms=None):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?

        '''

        self.transforms = torch.nn.Sequential(*transforms) if transforms is not None else None

        # Load the directory and all files in them

        self.mfcc_dir = os.path.join(root + "/" + partition + "/mfcc/")
        self.transcript_dir = os.path.join(root + "/" + partition + "/transcript/")

        self.mfcc_files = [os.path.join(self.mfcc_dir + file) for file in sorted(os.listdir(self.mfcc_dir))]
        self.transcript_files = [os.path.join(self.transcript_dir + file) for file in sorted(os.listdir(self.transcript_dir))]

        self.phonemes = phonemes

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        assert len(self.mfcc_files) == len(self.transcript_files)
        self.length = len(self.mfcc_files)

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS


        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
        self.mfccs = []
        self.transcripts = []

        for i in tqdm(range(len(self.mfcc_files)), desc=f"Creating {partition}"):
            mfcc_file = self.mfcc_files[i]
            transcript_file = self.transcript_files[i]

            mfcc = np.load(mfcc_file)
            # normalize
            mfcc = (mfcc - mfcc.mean(axis=0, keepdims=True)) / (mfcc.std(axis=0, keepdims=True) + 1e-5)

            transcript  = np.load(transcript_file)[1:-1]

            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        for i in range(len(self.transcripts)):
          self.transcripts[i] = [phonemes.index(phoneme) for phoneme in self.transcripts[i]]


    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        mfcc = torch.FloatTensor(self.mfccs[ind]) # TODO
        transcript = torch.tensor(self.transcripts[ind]) # TODO

        return mfcc, transcript


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lengths of features,
            and lengths of labels.
        '''

        #######################################################################
        """
        NOTE: What is the collate_fn? https://www.youtube.com/watch?v=5hH3CVzy8zs

        The collate_fn parameter in a PyTorch DataLoader allows you to control how
        individual data samples are combined into a batch. By default, the DataLoader
        stacks the data samples along the first dimension to form a batch. However,
        not all data can be neatly stacked this way (think about images of different
        sizes or text sequences of varying lengths). Here's where collate_fn shines—it's
        your Swiss Army knife for handling these quirks elegantly.

        https://plainenglish.io/blog/understanding-collate-fn-in-pytorch-f9d1742647d3
        collate essentially allows you to

        """
        #######################################################################

        # batch of input mfcc coefficients
         # TODO
        # batch of output phonemes
         # TODO
        batch_mfcc = [] # TODO
        batch_transcript = [] # TODO
        for mfcc, transcript in batch:
            batch_mfcc.append(mfcc)
            batch_transcript.append(transcript)

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = torch.nn.utils.rnn.pad_sequence(batch_mfcc, batch_first=True) # TODO
        lengths_mfcc = [len(mfcc) for mfcc in batch_mfcc] # TODO

        batch_transcript_pad = torch.nn.utils.rnn.pad_sequence(batch_transcript, batch_first=True) # TODO
        lengths_transcript = [len(transcript) for transcript in batch_transcript] # TODO

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?

        if self.transforms is not None:
            batch_mfcc_pad = self.transforms(batch_mfcc_pad)
            # batch_transcript_pad = self.transform(batch_transcript_pad)


        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)



### Test Data

In [ ]:
class AudioDatasetTest(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root, phonemes=PHONEMES, partition="test-clean"):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = os.path.join(root + "/" + partition + "/mfcc/")

        self.mfcc_files = [os.path.join(self.mfcc_dir + file) for file in sorted(os.listdir(self.mfcc_dir))]

        self.phonemes = phonemes

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS


        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
        self.mfccs = []

        for i in tqdm(range(len(self.mfcc_files)), desc=f"Creating {partition}"):
            mfcc_file = self.mfcc_files[i]

            mfcc = np.load(mfcc_file)
            # normalize
            mfcc = (mfcc - mfcc.mean(axis=0, keepdims=True)) / (mfcc.std(axis=0, keepdims=True) + 1e-5)

            self.mfccs.append(mfcc)

    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        mfcc = torch.FloatTensor(self.mfccs[ind]) # TODO

        return mfcc


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lengths of features,
            and lengths of labels.
        '''

        #######################################################################
        """
        NOTE: What is the collate_fn? https://www.youtube.com/watch?v=5hH3CVzy8zs

        The collate_fn parameter in a PyTorch DataLoader allows you to control how
        individual data samples are combined into a batch. By default, the DataLoader
        stacks the data samples along the first dimension to form a batch. However,
        not all data can be neatly stacked this way (think about images of different
        sizes or text sequences of varying lengths). Here's where collate_fn shines—it's
        your Swiss Army knife for handling these quirks elegantly.

        https://plainenglish.io/blog/understanding-collate-fn-in-pytorch-f9d1742647d3
        collate essentially allows you to

        """
        #######################################################################

        # batch of input mfcc coefficients
         # TODO
        # batch of output phonemes
         # TODO
        batch_mfcc = [] # TODO
        for mfcc in batch:
            batch_mfcc.append(mfcc)

        batch_mfcc_pad = torch.nn.utils.rnn.pad_sequence(batch_mfcc, batch_first=True) # TODO
        lengths_mfcc = [len(mfcc) for mfcc in batch_mfcc] # TODO

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)



### Config - Hyperparameters

### Data loaders

In [ ]:
# get me RAMMM!!!!
import gc
gc.collect()

1191

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset(root, transforms=transforms)
val_data = AudioDataset(root, partition="dev-clean")
test_data = AudioDatasetTest(root)

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True,
    collate_fn  = train_data.collate_fn
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False,
    collate_fn  = val_data.collate_fn
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False,
    collate_fn  = test_data.collate_fn
)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Creating test-clean: 100%|██████████| 2620/2620 [00:01<00:00, 2548.93it/s]

Batch size:  128
Train dataset samples = 28539, batches = 223
Val dataset samples = 2703, batches = 22
Test dataset samples = 2620, batches = 21


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([128, 1662, 27]) torch.Size([128, 191]) torch.Size([128]) torch.Size([128])


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [ ]:
# torch.cuda.empty_cache()

# class Network(nn.Module):

#     def __init__(self, out_feats=41):

#         super(Network, self).__init__()

#         # Adding some sort of embedding layer or feature extractor might help performance.
#         # self.embedding = ?

#         # TODO : look up the documentation. You might need to pass some additional parameters.
#         self.lstm = nn.LSTM(input_size = __, hidden_size = 256, num_layers = 1)

#         self.classification = nn.Sequential(
#             #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
#         )


#         self.logSoftmax = #TODO: Apply a log softmax here. Which dimension would apply it on ?

#     def forward(self, x, lx):
#         #TODO
#         # The forward function takes 2 parameter inputs here. Why?
#         # Refer to the handout for hints
#         pass

## Initialize Basic Network
(If trying out the basic Network)

In [ ]:
# torch.cuda.empty_cache()

# model = Network().to(device)
# summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

## ASR Network

### Pyramid Bi-LSTM (pBLSTM)

In [ ]:
# del model
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [ ]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input?
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''

    def __init__(self, input_size, hidden_size, layers=1, dropout=0.0):
        super(pBLSTM, self).__init__()

        self.blstm = torch.nn.LSTM(input_size, hidden_size,
                                   batch_first=True, bidirectional=True,
                                   dropout=dropout, num_layers=layers)
        # TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size

    def forward(self, x_packed): # x_packed is a PackedSequence

        # TODO: Pad Packed Sequence

        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        # TODO: Pack Padded Sequence. What output(s) would you get?
        # TODO: Pass the sequence through bLSTM

        x, x_lens = nn.utils.rnn.pad_packed_sequence(x_packed, batch_first=True)

        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        x, x_lens = self.trunc_reshape(x, x_lens)

        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        # Pack Padded Sequence. What output(s) would you get?
        x_packed = nn.utils.rnn.pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)


        out, _ = self.blstm(x_packed)

        return out

    def trunc_reshape(self, x, x_lens):
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        # TODO: Reduce lengths by the same downsampling factor

        B, L, D = x.shape

        # get rid of 1 if length is odd
        if L % 2 != 0:
          x = x[:, :-1, :]

        # per pyramidal, reduce length to increase computation speed
        x = x.reshape(B, L // 2, D*2)
        x_lens = x_lens // 2


        return x, x_lens

### Encoder

In [ ]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Encoder, self).__init__()

        '''
        the purpose of this is to increase the dimensionality to express
        27 channels more effectively.

        you can also increase stride here to downsample the input to increase comp speed.
        Dont do too much or else you will rid important information from the input.

        '''

        self.embedding = torch.nn.Sequential(
            PermuteBlock(),
            torch.nn.Conv1d(input_size, encoder_hidden_size//3, kernel_size=5, padding=5//2),
            torch.nn.BatchNorm1d(encoder_hidden_size//3),
            torch.nn.GELU(),
            torch.nn.Dropout1d(0.1),
            torch.nn.Conv1d(encoder_hidden_size//3, encoder_hidden_size//2, kernel_size=3, padding=3//2),
            torch.nn.BatchNorm1d(encoder_hidden_size//2),
            torch.nn.GELU(),
            torch.nn.Dropout1d(0.2),
            torch.nn.Conv1d(encoder_hidden_size//2, encoder_hidden_size, kernel_size=3, padding=3//2),
            torch.nn.BatchNorm1d(encoder_hidden_size),
            torch.nn.GELU(),
            torch.nn.Dropout1d(0.1),
            torch.nn.Conv1d(encoder_hidden_size, encoder_hidden_size, kernel_size=2, padding=2//2),
            torch.nn.BatchNorm1d(encoder_hidden_size),
            torch.nn.GELU(),
            PermuteBlock(),
        )

         #TODO: You can use CNNs as Embedding layer to extract features. Keep in mind the Input dimensions and expected dimension of Pytorch CNN.

        self.pBLSTMs = torch.nn.Sequential( # How many pBLSTMs are required?
            # TODO: Fill this up with pBLSTMs - What should the input_size be?
            # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
            # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
            # https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py#L5
            # ...
            # ...
            pBLSTM(encoder_hidden_size*2, encoder_hidden_size, layers=2, dropout=0.15),
            pBLSTM(encoder_hidden_size*4, encoder_hidden_size, layers=3, dropout=0.20),
        )

    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        #TODO: Call the embedding layer
        # TODO: Pack Padded Sequence
        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
        # TODO: Pad Packed Sequence

        x = self.embedding(x)
        x_packed = nn.utils.rnn.pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        out = self.pBLSTMs(x_packed)
        encoder_outputs, encoder_lens = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        # Remember the number of output(s) each function returns

        return encoder_outputs, encoder_lens

### Decoder

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            #TODO define your MLP arch. Refer HW1P2
            #Use Permute Block before and after BatchNorm1d() to match the size
            # PermuteBlock(),
            # torch.nn.BatchNorm1d(embed_size),
            # PermuteBlock(),
            torch.nn.Linear(embed_size, embed_size),
            PermuteBlock(),
            torch.nn.BatchNorm1d(embed_size),
            PermuteBlock(),
            torch.nn.GELU(),
            torch.nn.Dropout(0.15),
            torch.nn.Linear(embed_size, embed_size//2),
            PermuteBlock(),
            torch.nn.BatchNorm1d(embed_size//2),
            PermuteBlock(),
            torch.nn.GELU(),
            torch.nn.Dropout(0.15),
            torch.nn.Linear(embed_size//2, embed_size//2),
            PermuteBlock(),
            torch.nn.BatchNorm1d(embed_size//2),
            PermuteBlock(),
            torch.nn.GELU(),
            torch.nn.Linear(embed_size//2, output_size)
        )

        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        #TODO call your MLP
        #TODO Think what should be the final output of the decoder for the classification

        out = self.mlp(encoder_out)
        out = self.softmax(out)

        return out

In [ ]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 192, output_size= len(PHONEMES)):
        super().__init__()

        self.augmentations  = torch.nn.Sequential(
            #TODO Add Time Masking/ Frequency Masking
            #Hint: See how to use PermuteBlock() function defined above
        )
        self.encoder        = Encoder(input_size, embed_size)# TODO: Initialize Encoder
        self.decoder        = Decoder(embed_size*2, output_size) # TODO: Initialize Decoder



    def forward(self, x, lengths_x):

        # if self.training:
        #     x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## Initialize ASR Network

In [ ]:
model = ASRModel(
    input_size  = 27,
    embed_size  = config["embed_size"],
    output_size = len(PHONEMES)
).to(device)
print(model)
summary(model, x.to(device), lx)

ASRModel(
  (augmentations): Sequential()
  (encoder): Encoder(
    (embedding): Sequential(
      (0): PermuteBlock()
      (1): Conv1d(27, 73, kernel_size=(5,), stride=(1,), padding=(2,))
      (2): BatchNorm1d(73, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): GELU(approximate='none')
      (4): Dropout1d(p=0.1, inplace=False)
      (5): Conv1d(73, 110, kernel_size=(3,), stride=(1,), padding=(1,))
      (6): BatchNorm1d(110, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (7): GELU(approximate='none')
      (8): Dropout1d(p=0.2, inplace=False)
      (9): Conv1d(110, 220, kernel_size=(3,), stride=(1,), padding=(1,))
      (10): BatchNorm1d(220, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (11): GELU(approximate='none')
      (12): Dropout1d(p=0.1, inplace=False)
      (13): Conv1d(220, 220, kernel_size=(2,), stride=(1,), padding=(1,))
      (14): BatchNorm1d(220, eps=1e-05, momentum=0.1, affine=True, track_run

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_encoder.embedding.PermuteBlock_0,-,"[128, 27, 1662]",NaN,NaN
1_encoder.embedding.Conv1d_1,"[27, 73, 5]","[128, 73, 1662]",9928.0,16379010.0
2_encoder.embedding.BatchNorm1d_2,[73],"[128, 73, 1662]",146.0,73.0
3_encoder.embedding.GELU_3,-,"[128, 73, 1662]",NaN,NaN
4_encoder.embedding.Dropout1d_4,-,"[128, 73, 1662]",NaN,NaN
5_encoder.embedding.Conv1d_5,"[73, 110, 3]","[128, 110, 1662]",24200.0,40037580.0
6_encoder.embedding.BatchNorm1d_6,[110],"[128, 110, 1662]",220.0,110.0
7_encoder.embedding.GELU_7,-,"[128, 110, 1662]",NaN,NaN
8_encoder.embedding.Dropout1d_8,-,"[128, 110, 1662]",NaN,NaN


# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [ ]:
#TODO

criterion = torch.nn.CTCLoss(blank=0)# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), config["lr"]) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(
    PHONEMES,
    model_path=None,
    alpha=0,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=1.0,
    beam_width=config["beam_width"],
    num_processes=4,
    blank_id=0,
    log_probs_input=True
)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones= config["milestones"], gamma= config["gamma"])

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

# Decode Prediction

In [ ]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):

    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    # print(f"{output.shape=}")
    # output = output.permute(1, 0, 2)
    # print(f"{output.shape=}")
    beam_results, beam_scores, timesteps, out_lens = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

    pred_strings = []

    for i in range(output_lens.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        pred_strings.append("".join([PHONEME_MAP[n] for n in beam_results[i][0][:out_lens[i][0]]]))

    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers

    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i]
        label_string = "".join([PHONEME_MAP[n] for n in label[i]])
        label_string = label_string.replace(" ", "")

        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    # raise NotImplemented
    return dist

# Test Implementation

In [ ]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    print("Levenshtein Dist: ", calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lx, ly, decoder, LABELS))
    # print("Levenshtein Dist: ", calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

torch.Size([128, 734, 41])
torch.Size([734, 128, 41]) torch.Size([128, 265])
tensor(7.1344, device='cuda:0', grad_fn=<MeanBackward0>)
Levenshtein Dist:  205.2265625


# WandB

You will need to fetch your api key from wandb.ai

In [ ]:
import wandb
wandb.login(key="58cec1d955c8a558ddb5374428c04433fd0455c6")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name = config['model'], ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

wandb: Currently logged in as: atrela (atrela-cmu). Use `wandb login --relogin` to force relogin


# Train Functions

In [ ]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

## Training Setup

In [ ]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    if metric:
      metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions

SAVE_DIR_ARCH = "/content/drive/MyDrive/s24-idl/hw3p2/Alec/archs"
SAVE_DIR_CHECK = "/content/drive/MyDrive/s24-idl/hw3p2/Alec/checkpoints"
SAVE_DIR_BEST = "/content/drive/MyDrive/s24-idl/hw3p2/Alec/bests"

if not os.path.exists(SAVE_DIR_ARCH) or not os.path.exists(SAVE_DIR_CHECK) or not os.path.exists(SAVE_DIR_BEST):
    raise Exception("One of the dirs does not exist")

### Save it in a txt file
model_arch  = str(model)
arch_file   = open(os.path.join(SAVE_DIR_ARCH + f"/{config['model']}.txt"), "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = 5.5 # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = os.path.join(SAVE_DIR_CHECK + f"/{config['model']}") #TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = os.path.join(SAVE_DIR_BEST + f"/{config['model']}") #TODO set best model path

In [ ]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(start, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = optimizer.param_groups[0]['lr']

    train_loss              = train_model(model, train_loader, criterion, optimizer)
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder)
    # scheduler.step(valid_dist)
    scheduler.step()

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch,
               f"{epoch_model_path}_e{epoch}.pth")
    wandb.save(f"{epoch_model_path}_e{epoch}.pth")
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch,
                   f"{best_model_path}_e{epoch}.pth")
        wandb.save(f"{best_model_path}_e{epoch}.pth")
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/50


	Train Loss 2.9592	 Learning Rate 0.0015000
	Val Dist 44.4525%	 Val Loss 1.9235


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Saved epoch model

Epoch: 2/50


	Train Loss 1.5247	 Learning Rate 0.0015000
	Val Dist 22.0630%	 Val Loss 1.0010
Saved epoch model

Epoch: 3/50


...

	Train Loss 0.2542	 Learning Rate 0.0000025
	Val Dist 5.3299%	 Val Loss 0.2771
Saved epoch model

Epoch: 48/50


	Train Loss 0.2464	 Learning Rate 0.0000025
	Val Dist 5.2767%	 Val Loss 0.2745
Saved epoch model

Epoch: 49/50


	Train Loss 0.2621	 Learning Rate 0.0000025
	Val Dist 5.2671%	 Val Loss 0.2738
Saved epoch model

Epoch: 50/50


	Train Loss 0.2347	 Learning Rate 0.0000025
	Val Dist 5.3036%	 Val Loss 0.2744
Saved epoch model


lr,████████████▄▄▄▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▄▃▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_dist,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,0.0
train_loss,0.2347
valid_dist,5.30365
valid_loss,0.27443


# Generate Predictions and Submit to Kaggle

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 125 #TODO

test_decoder    = CTCBeamDecoder(
    PHONEMES,
    model_path=None,
    alpha=0,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=1.0,
    beam_width=TEST_BEAM_WIDTH,
    num_processes=4,
    blank_id=0,
    log_probs_input=True
)#TODO

results = []

LOAD_PATH = "/content/drive/MyDrive/s24-idl/hw3p2/Alec/bests/benchmark-more-MORE-pBLSTMs-transf_e37.pth"
if LOAD_PATH:
  model, *_ = load_model(LOAD_PATH, model, metric=None)

# model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    prediction_string = decode_prediction(h, lh, test_decoder)# TODO call decode_prediction
    #TODO save the output in results array.
    for result in prediction_string:
      results.append(result)

    del x, lx, h, lh
    torch.cuda.empty_cache()

Testing


100%|██████████| 21/21 [02:02<00:00,  5.85s/it]


In [ ]:
data_dir = f"{root}/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv(f"/content/drive/MyDrive/s24-idl/hw3p2/Alec/subs/final_{config['model']}_submission.csv", index = False)

In [ ]:
!kaggle competitions submit -c hw3p2asr-s24 -f submission.csv -m "I made it!"